In [1]:
# Load in data and clean corpus
# So, basically, what I should have done was read and clean the data in another file, then load the cleaned data
# Problem: my ongoing incompetence.
# So instead I just put all the re-used code (shamelessly copied from the illustrious Mr Agnihotri) in here instead.
# YAY
import numpy as np
import codecs
import nltk
import re
from nltk.tokenize import wordpunct_tokenize
from nltk import PorterStemmer
class Document():
    
    """ The Doc class rpresents a class of individual documents
    
    """
    
    def __init__(self, speech_year, speech_pres, speech_text):
        self.year = speech_year
        self.pres = speech_pres
        self.text = speech_text.lower()
        self.tokens = np.array(wordpunct_tokenize(self.text))
        
        
        
    def token_clean(self,length):

        """ 
        description: strip out non-alpha tokens and tokens of length > 'length'
        input: length: cut off length 
        """

        self.tokens = np.array([t for t in self.tokens if (t.isalpha() and len(t) > length)])

    def stopword_remove(self, stopwords):

        """
        description: Remove stopwords from tokens.
        input: stopwords: a suitable list of stopwords
        """

        
        self.tokens = np.array([t for t in self.tokens if t not in stopwords])


    def stem(self):

        """
        description: Stem tokens with Porter Stemmer.
        """
        
        self.tokens = np.array([PorterStemmer().stem(t) for t in self.tokens])
class Corpus():
    
    """ 
    The Corpus class represents a document collection
     
    """
    def __init__(self, doc_data, stopword_file, clean_length):
        """
        Notice that the __init__ method is invoked everytime an object of the class
        is instantiated
        """
        

        #Initialise documents by invoking the appropriate class
        self.docs = [Document(doc[0], doc[1], doc[2]) for doc in doc_data] 
        
        self.N = len(self.docs)
        self.clean_length = clean_length
        
        #get a list of stopwords
        self.create_stopwords(stopword_file, clean_length)
        #stopword removal, token cleaning and stemming to docs
        self.clean_docs(2)
        
        #create vocabulary
        self.corpus_tokens()
        
    def clean_docs(self, length):
        """ 
        Applies stopword removal, token cleaning and stemming to docs
        """
        for doc in self.docs:
            doc.token_clean(length)
            doc.stopword_remove(self.stopwords)
            doc.stem()        
    
    def create_stopwords(self, stopword_file, length):
        """
        description: parses a file of stowords, removes words of length 'length' and 
        stems it
        input: length: cutoff length for words
               stopword_file: stopwords file to parse
        """
        
        with codecs.open(stopword_file,'r','utf-8') as f: raw = f.read()
        
        self.stopwords = (np.array([PorterStemmer().stem(word) 
                                    for word in list(raw.splitlines()) if len(word) > length]))
    def corpus_tokens(self):
        """
        description: create a set of all all tokens or in other words a vocabulary
        """
        
        #initialise an empty set
        self.token_set = set()
        for doc in self.docs:
            self.token_set = self.token_set.union(doc.tokens) 
            
def parse_text(textraw, regex):
    """takes raw string and performs two operations
    1. Breaks text into a list of speech, president and speech
    2. breaks speech into paragraphs
    """
    prs_yr_spch_reg = re.compile(regex, re.MULTILINE|re.DOTALL)
    
    #Each tuple contains the year, last ane of the president and the speech text
    prs_yr_spch = prs_yr_spch_reg.findall(textraw)
    
    #convert immutabe tuple to mutable list
    prs_yr_spch = [list(tup) for tup in prs_yr_spch]
    
    for i in range(len(prs_yr_spch)):
        prs_yr_spch[i][2] = prs_yr_spch[i][2].replace('\n', '')
    
    #sort
    prs_yr_spch.sort()
    
    return(prs_yr_spch)

text = open('./../data/pres_speech/sou_all.txt', 'r').read()
regex = "_(\d{4}).*?_[a-zA-Z]+.*?_[a-zA-Z]+.*?_([a-zA-Z]+)_\*+(\\n{2}.*?)\\n{3}"
pres_speech_list = parse_text(text, regex)
#Instantite the corpus class
corpus = Corpus(pres_speech_list, './../data/stopwords/stopwords.txt', 2)
for doc in corpus.docs:
    if len(doc.text) < 500:
        print doc
        print len(doc.text)

In [2]:
# BUT LOOK I WROTE THIS BIT MYSELF.
#it is 3 lines long
corpus_forLDA = []
for doc in corpus.docs:
    corpus_forLDA.append(doc.tokens[0:400])
    
vocabulary = set(corpus_forLDA[0])
for doc in range(len(corpus_forLDA)):
    vocabulary = vocabulary.union(set(corpus_forLDA[doc]))
vocabulary = list(vocabulary)
# Set aspects of the corpus: D is n.doc, N is n.words, k is n.topics, V is len.vocabulary, w is a 
D = len(corpus_forLDA)
k = 50
V = len(vocabulary)
N = len(corpus_forLDA[0])
w = np.array([])
for doc in range(D):
    w = np.append(w, corpus_forLDA[doc])
    
w = np.reshape(w, (D, N))   
set(corpus_forLDA[1])


{u'abroad',
 u'access',
 u'accord',
 u'administr',
 u'admit',
 u'adopt',
 u'advanc',
 u'affair',
 u'afford',
 u'aggressor',
 u'agre',
 u'agricultur',
 u'aid',
 u'allow',
 u'alreadi',
 u'among',
 u'answer',
 u'appoint',
 u'arm',
 u'arrang',
 u'ascertain',
 u'attend',
 u'attent',
 u'auspici',
 u'author',
 u'avoid',
 u'basi',
 u'befor',
 u'best',
 u'better',
 u'bless',
 u'burthen',
 u'call',
 u'can',
 u'carolina',
 u'certain',
 u'cherish',
 u'circumst',
 u'citizen',
 u'comfort',
 u'commerc',
 u'common',
 u'commonwealth',
 u'commun',
 u'compens',
 u'compet',
 u'comprehend',
 u'concili',
 u'concord',
 u'conduc',
 u'conduct',
 u'confid',
 u'congratul',
 u'consider',
 u'constitu',
 u'constitut',
 u'consult',
 u'contain',
 u'contribut',
 u'conveni',
 u'convinc',
 u'cool',
 u'countri',
 u'cours',
 u'credit',
 u'currenc',
 u'deem',
 u'defens',
 u'defin',
 u'defray',
 u'degre',
 u'deliber',
 u'depred',
 u'deriv',
 u'deserv',
 u'design',
 u'desir',
 u'difficulti',
 u'digest',
 u'direct',
 u'discer

In [9]:
# Determine initial allocation
from collections import Counter

# Set the hyperparameters
alpha = np.array([50/k] * k)
eta = [200/float(V)] * V

# Initialise the document-specific mixing probabilities, D x k
theta = np.array([])
for doc in range(D):
    theta = np.append(theta, np.random.dirichlet(alpha))
theta = np.reshape(theta, (D, k))

# Initialise the topic allocations, D x k - that is, in document d topic k occured this many times
z = np.array([])
for doc in range(D):
    z = np.append(z, np.random.multinomial(N, pvals = theta[doc], size = 1))
    
z = np.reshape(z, (D, k))

# Initialise the topic-specific term probabilities
beta = np.array([])
for topic in range(k):
    beta = np.append(beta, np.random.dirichlet(eta))
beta = np.reshape(beta, (k, V))


In [11]:
eta

[0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.03208727739451307,
 0.0320872

In [13]:
# Start iterating
it = 1
total_results = []
beta_collect = []
while it < 20:
    print it
    
    # Draw the new theta values
    alpha_new = z + alpha
    theta = np.array([])
    for doc in range(D):
        theta = np.append(theta, np.random.dirichlet(alpha_new[doc]))
    theta = np.reshape(theta, (D, k))
       
    # Draw the new beta values
    # Find the ms
    count_vocab = dict(zip(vocabulary, [0] * V)) 
    count_terms = Counter(w[0])
    for key in count_vocab:
        count_vocab[key] += count_terms[key]
    m = np.array(count_vocab.values())
    eta_new = m + eta
    # Find the betas
    beta = np.array([])
    for topic in range(k):
        beta = np.append(beta, np.random.dirichlet(eta_new))
    beta = np.reshape(beta, (k, V))
    beta_collect.append(beta)
    
    # Draw the new zs
    z_new = np.array([])
    for doc in range(D):
        # Our first task is to find the document-specific mixing probabilities
        beta_theta2 = np.matrix(beta)
        beta_theta2 = np.transpose(beta_theta2)
        denominators = sum(beta_theta2)
        # First we find a N x k matrix of beta[k,n] *theta[d,k] 
        beta_theta = np.array([])
        denominator = [0] * V
        for row in range(len(beta)):
            be = beta[row] * theta[doc, row]
            denominator += be
            beta_theta = np.append(beta_theta, be)

        # We get this into a kxN matrix, because reasons
        beta_theta = np.reshape(beta_theta, (k, V))
        beta_theta = np.matrix(beta_theta)
        beta_theta = np.transpose(beta_theta)

        # And then we simulate the z values
        z_thisdoc = np.array([0] * k)
        for word in range(len(w[doc])):
            which_vocab = vocabulary.index(w[doc,word])
            this_mix = beta_theta[which_vocab] / denominator[which_vocab]
            z_thisdoc = z_thisdoc + np.random.multinomial(1, pvals = this_mix.getA1())
        z_new = np.append(z_new, z_thisdoc)
    z_new = np.reshape(z_new, (D, k))
    z = z_new.copy()
    
    # Every 50th iteration, we take a sample
    if it > 1000 & it % 50 == 0:
        results = []
        results.append(beta)
        for topic in range(k):
            top5 = dict(zip(vocabulary, beta[topic]))
            new_top5 = dict(sorted(top5.iteritems(), key=operator.itemgetter(1), reverse=True)[:5])
            results.append(new_top5)


        total_results.append(results)
    
    it += 1

print total_results[0]

1
2


KeyboardInterrupt: 

In [32]:
import operator
beta
print len(beta)
print len(beta[0])
results = []
for topic in range(k):
    top5 = dict(zip(vocabulary, beta[topic]))
    new_top5 = dict(sorted(top5.iteritems(), key=operator.itemgetter(1), reverse=True)[:5])
    results.append(new_top5)

results[49]




50
6233


{u'gallantri': 0.0091858850040500376,
 u'groundwork': 0.0097271971942260241,
 u'guest': 0.012334668709504436,
 u'pit': 0.010651144751761869,
 u'year': 0.0091818588686582003}

In [75]:
count_vocab = dict(zip(vocabulary, [0] * V)) 
count_terms = Counter(w[0])
for key in count_vocab:
    count_vocab[key] += count_terms[key]
m = np.array(count_vocab.values())
eta_new = m + eta
print len(eta)
vocabulary[0:5]

6233


[u'tripolitan', u'affairsw', u'yellow', u'interchang', u'elvi']

In [66]:
beta = np.array([])
for topic in range(k):
    print topic
    beta = np.append(beta, np.random.dirichlet(eta_new))
    print len(beta)
beta = np.reshape(beta, (k, V))
beta_collect.append(beta)

0
6233
1
12466
2
18699
3
24932
4
31165
5
37398
6
43631
7
49864
8
56097
9
62330
10
68563
11
74796
12
81029
13
87262
14
93495
15
99728
16
105961
17
112194
18
118427
19
124660
20
130893
21
137126
22
143359
23
149592
24
155825
25
162058
26
168291
27
174524
28
180757
29
186990
30
193223
31
199456
32
205689
33
211922
34
218155
35
224388
36
230621
37
236854
38
243087
39
249320
40
255553
41
261786
42
268019
43
274252
44
280485
45
286718
46
292951
47
299184
48
305417
49
311650
